# コンペ用Fine-Tuningテンプレート

こちらは、コンペにてFineーTuningを行いたい方に向けたテンプレートとなるFine-tuningコードです。
こちらを実行いただくだけでコンペの基準に達することができると思います。上手く活用してコンペ上位を目指しましょう!!

本コードはOmnicampusで提供される演習環境での実行を想定しています。  
それ以外の環境で実行される場合は適宜、修正して下さい。  

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
)
import os, torch, gc
from datasets import load_dataset
import bitsandbytes as bnb
from trl import SFTTrainer

In [31]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [37]:
# Hugging Face Token
HF_TOKEN = os.environ.get("HF_TOKEN")

In [7]:
from pathlib import Path
# モデルを読み込み。
# llm-jp-3 1.8B, 3.7B, 13Bのsnapshotをダウンロード済みでmodelsディレクトリに格納してあります。
# base_model_idの値はomnicampusの環境におけるモデルのパスを表しており、それ以外の環境で実行する場合は変更の必要があります。
# その他のモデルは取得に承諾が必要なため、各自でダウンロードお願いします。

base_model_id = Path("../models/llm-jp/llm-jp-3-1.8b")  # Fine-Tuningするベースモデル
# omnicampus以外の環境をご利用の方は以下をご利用ください。
# base_model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "../models/llm-jp-3-1-8b-finetune"  # Fine-Tuningしたモデルにつけたい名前

In [8]:
"""
bnb_config: 量子化の設定

  - load_in_4bit:
      - 4bit量子化形式でモデルをロード

  - bnb_4bit_quant_type:
      - 量子化の形式を指定

  - bnb_4bit_compute_dtype:
      - 量子化された重みを用いて計算する際のデータ型

"""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # nf4は通常のINT4より精度が高く、ニューラルネットワークの分布に最適です
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [9]:
"""
model: モデル

  - base_model:
      - 読み込むベースモデル (事前に定義したもの)

  - quantization_config:
      - bnb_configで設定した量子化設定

  - device_map:
      - モデルを割り当てるデバイス (CPU/GPU) "auto"で自動に割り当てられます。

tokenizer: トークナイザー

  - base_model:
      - 読み込むベースモデル (事前に定義したもの)

  - trust_remote_code:
      - リモートコードの実行を許可 (カスタムモデルなど)
"""
model = AutoModelForCausalLM.from_pretrained(
    base_model_id, quantization_config=bnb_config, device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [10]:
"""
find_all_linear_names: モデル内の4bit量子化線形層を探します。
"""


def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit  # 4bit量子化線形層クラスを指定
    lora_module_names = set()  # ここに取得した線形層を保持します。

    # モデル内の全てのモジュールを探索します
    for name, module in model.named_modules():
        if isinstance(module, cls):  # モジュールが4bit量子化線形層の場合
            names = name.split(
                "."
            )  # モジュールの名前を分割 (ネストされてる際などに対処)
            lora_module_names.add(
                names[0] if len(names) == 1 else names[-1]
            )  # 最下層の名前をlora_module_namesに追加

    # 'lm_head' は16ビット演算の際に除外する必要があるため、lora_module_namesから削除
    if "lm_head" in lora_module_names:
        lora_module_names.remove("lm_head")

    return list(lora_module_names)  # lora_module_namesをリストに変換して返します。


modules = find_all_linear_names(model)

In [11]:
"""
peft_config: PEFTの構成設定

  - r
      - LoRA のランク (4, 8, 16 ,32...)
      - 増やすほど学習が捗るが, 過学習のリスクも高まるので注意

  - lora_alpha
      - LoRAのスケーリング係数

  - lora_dropout
      - ドロップアウト率（過学習を防ぐための割合）

  - bias
      - バイアス項の扱い ("none"の場合、LoRAはバイアスを学習しない)

  - task_type
      - タスクタイプ

  - target_modules
      - LoRAを適用するターゲットモジュール (前のコードで特定した層)
"""

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

model = get_peft_model(model, peft_config)

In [12]:
"""
学習に用いるデータセットの指定
今回はLLM-jp の公開している Ichikara Instruction を使います。データにアクセスするためには申請が必要ですので、使いたい方のみ申請をしてください。
Ichikara Instruciton を Hugging Face Hub にて公開することはお控えください。
また、CC-BY-NC-SAですのでモデルはライセンスを継承する前提でお使いください。

下記のリンクから申請を終えた先に Google Drive があり、Distribution20241221_all というフォルダごとダウンロードしてください。
今回は「ichikara-instruction-003-001-1.json」を使います。必要であれば展開（!unzip など）し、データセットのパスを適切に指定してください。
omnicampusの開発環境では取得したデータを左側にドラッグアンドドロップしてお使いください。

https://liat-aip.sakura.ne.jp/wp/llmのための日本語インストラクションデータ作成/llmのための日本語インストラクションデータ-公開/
関根聡, 安藤まや, 後藤美知子, 鈴木久美, 河原大輔, 井之上直也, 乾健太郎. ichikara-instruction: LLMのための日本語インストラクションデータの構築. 言語処理学会第30回年次大会(2024)

"""

dataset = load_dataset(
    "json",
    data_files="../data/Distribution20241221_all/ichikara-instruction-003-001-1.json",
)
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'output'],
        num_rows: 1729
    })
})

In [13]:
# 学習時のプロンプトフォーマットの定義
prompt = """### 指示
{}
### 回答
{}"""


"""
formatting_prompts_func: 各データをプロンプトに合わせた形式に合わせる
"""
EOS_TOKEN = tokenizer.eos_token  # トークナイザーのEOSトークン（文末トークン）


def formatting_prompts_func(examples):
    input = examples["text"]  # 入力データ
    output = examples["output"]  # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN  # プロンプトの作成
    return {
        "formatted_text": text,
    }  # 新しいフィールド "formatted_text" を返す


pass

# # 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc=4,  # 並列処理数を指定
)

dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'output', 'formatted_text'],
        num_rows: 1729
    })
})

In [14]:
# データを確認
print(dataset["train"]["formatted_text"][3])

### 指示
言葉の単位には、小さいものから順に「単語」→「文節」→「文」→「段落」→「文章」があります。
この5つの単位の違いを説明しなさい。
### 回答
日本語において、「言葉の単位」は5つあります。
1つずつ見ていくと、
・文章・・・1冊の本、1編の論文、1件のレポートなどはそれ全体で1つの文章ということになります。「言葉の単位」で最大の単位です。

・段落・・・「文章」を意味内容で区切ったひとまとまりのことを指します。日本語では、1字下げ（段落の先頭を1文字空ける）のルールがあります。

・文・・・句点「。」によって区切られるひと続きの言葉のことを言い、必ず文の終わりには「。」を付けます。

・文節・・・文を「意味の通じる最小の単位」まで区切ったものをいいます。
文：自分で薪を割れ、二重に温まる。
文節分け：自分で　/　薪を　/　割れ、　/　二重に　/　温まる。/

・単語・・・文節をさらに区切って、もうこれ以上分けることができない「言葉の単位として最小の単位」です。
単語分け：自分　/　で　/　薪　/　を　/　割れ、/　二重　/　に　/　温まる。/

となります。</s>


In [15]:
# データをtrainデータとtestデータに分割 (test_sizeの比率に)
# dataset = dataset["train"].train_test_split(test_size=0.1)
# dataset

In [16]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_
  _batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定（第8回演習を参考にすると良いです）

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""

training_arguments = TrainingArguments(
    output_dir=new_model_id,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_strategy="steps",
    logging_steps=10,
    warmup_steps=10,
    save_steps=100,
    save_total_limit=2,
    max_steps=-1,
    learning_rate=5e-5,
    fp16=False,
    bf16=False,
    seed=3407,
    group_by_length=True,
    report_to="none",
)

In [17]:
"""
SFTTrainer: Supervised Fine-Tuningに関する設定

  - model:
      - 読み込んだベースのモデル

  - train_dataset:
      - トレーニングに使用するデータセット

  - eval_dataset:
      - 評価に使用するデータセット

  - peft_config:
      - PEFT（Parameter-Efficient Fine-Tuning）の設定（LoRAを利用する場合に指定）

  - max_seq_length:
      - モデルに入力されるシーケンスの最大トークン長

  - dataset_text_field:
      - データセット内の学習に使うテキストを含むフィールド名

  - tokenizer:
      - モデルに対応するトークナイザー

  - args:
      - トレーニングに使用するハイパーパラメータ（TrainingArgumentsの設定を指定）

  - packing:
      - 入力シーケンスのパッキングを行うかどうかの設定 (False に設定することで、各入力を独立して扱う)
"""
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="formatted_text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

model.config.use_cache = False  # キャッシュ機能を無効化
tokenizer.pad_token = tokenizer.eos_token
trainer.train()  # トレーニングを実行

/mnt/c/Users/hiroga/Documents/GitHub/til/computer-science/machine-learning/_src/llm-exercises/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/mnt/c/Users/hiroga/Documents/GitHub/til/computer-science/machine-learning/_src/llm-exercises/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/mnt/c/Users/hiroga/Documents/GitHub/til/computer-science/machine-learning/_src/llm-exercises/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the S

Step,Training Loss
10,2.670100
20,2.476300
30,2.377700
40,2.328500
50,2.113400
60,2.273500
70,2.212500
80,2.139400
90,2.092600
100,2.088400


TrainOutput(global_step=864, training_loss=2.125037901931339, metrics={'train_runtime': 388.6213, 'train_samples_per_second': 4.449, 'train_steps_per_second': 2.223, 'total_flos': 3518033915363328.0, 'train_loss': 2.125037901931339, 'epoch': 0.9994216310005783})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if model and tokenizer are already defined
if "model" not in globals():
    model = AutoModelForCausalLM.from_pretrained(new_model_id, device_map="auto")

if "tokenizer" not in globals():
    tokenizer = AutoTokenizer.from_pretrained(new_model_id, trust_remote_code=True)

In [40]:
from datasets import load_dataset

ds = load_dataset("elyza/ELYZA-tasks-100")

In [19]:
# モデルによるタスクの推論。
from tqdm import tqdm

results = []
for i, data in enumerate(tqdm(ds["test"])):
    input = data["input"]

    prompt = f"""### 指示
  {input}
  ### 回答
  """

    tokenized_input = tokenizer.encode(
        prompt, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    attention_mask = torch.ones_like(tokenized_input)

    with torch.no_grad():
        outputs = model.generate(
            tokenized_input,
            attention_mask=attention_mask,
            max_new_tokens=100,
            do_sample=False,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id,
        )[0]
    output = tokenizer.decode(
        outputs[tokenized_input.size(1) :], skip_special_tokens=True
    )

    results.append({"task_id": i, "input": input, "output": output})

100%|██████████| 100/100 [11:46<00:00,  7.06s/it]


In [ ]:
# こちらで生成されたjsolを提出してください。
# 本コードではinputとeval_aspectも含んでいますが、なくても問題ありません。
# 必須なのはtask_idとoutputとなります。
import json
import re

jsonl_id = re.sub(".*/", "", new_model_id)
with open(f"../output/{jsonl_id}-outputs.jsonl", "w", encoding="utf-8") as f:
    for result in results:
        json.dump(
            result, f, ensure_ascii=False
        )  # ensure_ascii=False for handling non-ASCII characters
        f.write("\n")

In [39]:
# モデルとトークナイザーをHugging Faceにアップロード
model.push_to_hub(
    "llm-jp-3-1-8b-finetune", token=HF_TOKEN, private=True
)  # Online saving
tokenizer.push_to_hub(
    "llm-jp-3-1-8b-finetune", token=HF_TOKEN, private=True
)  # Online saving

adapter_model.safetensors:   0%|          | 0.00/67.7M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/xhiroga/llm-jp-3-1-8b-finetune/commit/0d807f5107865f045759a6b4c41aa8969f061d4a', commit_message='Upload tokenizer', commit_description='', oid='0d807f5107865f045759a6b4c41aa8969f061d4a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/xhiroga/llm-jp-3-1-8b-finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='xhiroga/llm-jp-3-1-8b-finetune'), pr_revision=None, pr_num=None)

https://liat-aip.sakura.ne.jp/wp/llmのための日本語インストラクションデータ作成/llmのための日本語インストラクションデータ-公開/

関根聡, 安藤まや, 後藤美知子, 鈴木久美, 河原大輔, 井之上直也, 乾健太郎. ichikara-instruction: LLMのための日本語インストラクションデータの構築. 言語処理学会第30回年次大会(2024)